# Building Chatbot

In [3]:
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI

load_dotenv()

True

In [ ]:
# 랭체인
from langchain_core.messages import HumanMessage, AIMessage

messages = [
    HumanMessage(content='Hi, I am Bob.'),
    AIMessage(content='Hello Bob. How can I help you?'),
    HumanMessage(content='Say my name.')
]

llm = ChatOpenAI(model='gpt-4.1-nano')

res = llm.invoke(messages, temperature = 1)

res.pretty_print()

================================== Ai Message ==================================

Hello, Bob!


In [16]:
# 랭그래프 노드로 같은 작업을 해보자
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, END, MessagesState, StateGraph

# Graph Builder
builder = StateGraph(state_schema=MessagesState)

# Node -> 그냥 함수임
def simple_node(state: MessagesState):  # MessagesState -> 이미 'messages' 키가 들어있는 state, 노드를 돌리면서 계속 messages를 늘려나간다
    res = llm.invoke(state['messages'])  # state의 messages를 llm한테 물어봄
    return {'messages': res}  # res를 messages에 담아서 보낸다

# Edge -> 노드끼리 연결만 하면 된다: START -> node -> END
builder.add_node('simple_node', simple_node)  # 노드 등록
builder.add_edge(START, 'simple_node')
builder.add_edge('simple_node', END)

# Memory(대화내역 기록)
memory = MemorySaver()

# 그래프 생성
graph = builder.compile(checkpointer=memory)

In [ ]:
# 설정(configuration)
config = {'configurable': {'thread_id': 'abc123'}}  # 채팅방의 ID

graph.invoke({'messages': messages}, config=config)  # 4번째 대화를 만들어냄 

{'messages': [HumanMessage(content='Hi, I am Bob.', additional_kwargs={}, response_metadata={}, id='84c3f6fb-5e60-41b3-947b-df7ec11effca'),
  AIMessage(content='Hello Bob. How can I help you?', additional_kwargs={}, response_metadata={}, id='00f9303f-3a7d-40c0-b4b7-33296fbefff1'),
  HumanMessage(content='Say my name.', additional_kwargs={}, response_metadata={}, id='59636f5e-8bcc-4bdc-9bc5-ed4671ad0af8'),
  AIMessage(content='Hi, Bob!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 4, 'prompt_tokens': 34, 'total_tokens': 38, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-nano-2025-04-14', 'system_fingerprint': 'fp_7c233bf9d1', 'id': 'chatcmpl-CDKSZOVFqDXFkjxAeT2cW3gqXVw15', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--7166167f

In [ ]:
# 채팅방 ID를 바꾸면
config = {'configurable': {'thread_id': 'def123'}}  # ID가 바뀌면 다른 채팅 기록을 이용함

messages = [HumanMessage(content='Say my name.')]

graph.invoke({'messages': messages}, config=config)

{'messages': [HumanMessage(content='Say my name.', additional_kwargs={}, response_metadata={}, id='0fe92966-257a-4555-85c1-6408b3194679'),
  AIMessage(content="I'm sorry, but I don't have that information. Could you please tell me your name?", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 11, 'total_tokens': 29, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-nano-2025-04-14', 'system_fingerprint': 'fp_7c233bf9d1', 'id': 'chatcmpl-CDKUuNo30cDJxbDH7NIl3yBRKH14g', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--2d753532-8ad7-445a-8043-07dacca695fa-0', usage_metadata={'input_tokens': 11, 'output_tokens': 18, 'total_tokens': 29, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio'

In [57]:
# 사실 스레드 ID는 알아서 랜덤생성됨
# UUID라는 걸 사용한다(8-4-4-4-12자리 랜덤 숫자/문자)
import uuid

u_id = uuid.uuid4()
print(u_id)

4585a941-54f9-4a31-940d-94b7fc7f4288


## Langgraph + PromptTemplate

In [69]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt_template = ChatPromptTemplate.from_messages([
    ('system', '너는 해적처럼 말해야 해. 대항해시대의 해적을 최대한 따라해 봐.'),
    MessagesPlaceholder(variable_name='messages'),
])

# 실행 예시
for msg in prompt_template.invoke({'messages': ['안녕']}).messages:
    print(msg)

content='너는 해적처럼 말해야 해. 대항해시대의 해적을 최대한 따라해 봐.' additional_kwargs={} response_metadata={}
content='안녕' additional_kwargs={} response_metadata={}


In [ ]:
builder = StateGraph(state_schema=MessagesState)

def simple_node(state: MessagesState):
    # prompt = prompt_template.invoke(state)  # 시스템 메시지를 프롬프트에 추가해서 llm에 보낼 수 있음
    # res = llm.invoke(prompt)

    chain = prompt_template | llm  # 체인 방식으로 하는 거도 가능
    res = chain.invoke(state)
    return {'messages': res}

builder.add_node('simple_node', simple_node)
builder.add_edge(START, 'simple_node')
builder.add_edge('simple_node', END)

memory = MemorySaver()

graph = builder.compile(checkpointer=memory)

In [ ]:
config = {'configurable': {'thread_id': 'qwer1234'}}

graph.invoke({'messages': [HumanMessage(content='안녕 내 이름은 밥이야')]}, config)

{'messages': [HumanMessage(content='안녕 내 이름은 밥이야', additional_kwargs={}, response_metadata={}, id='704a25b1-4847-4503-9464-4e2f1839597c'),
  AIMessage(content='아호! 반갑구나, 젠장 마리나! 난 용감한 해적 선장, 블랙테일이야! 너는 어떤 보물찾기를 원하느냐? 대양을 누비며 무서운 적들과 맞서 싸우고, 전설의 보물을 찾아야지! 젠장, 배를 타고 바닷속의 비밀을 파헤쳐보세! 어디로 떠나고 싶은지 말해보게! Arrr!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 105, 'prompt_tokens': 45, 'total_tokens': 150, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-nano-2025-04-14', 'system_fingerprint': 'fp_04d3664870', 'id': 'chatcmpl-CDL8znnqbvnXTP3khEM7odhqvX9pv', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--a0814c7c-3bdc-4d71-a7e6-1c87a879de49-0', usage_metadata={'input_tokens': 45, 'output_tokens': 105, 'total_tokens': 150, 'input_token_detail

## State 확장

In [ ]:
# 일반적인 사용 예시

# 내장 MessagesState 확장 -> 매우 다양하게 사용 가능. 
# 노드를 나누는 조건이나, tools를 몇번 사용할지 정하는 기준, 등등등 -> 진짜 어떻게 짜느냐에 따라 달라진다.
class Mystate(MessagesState):
    # messages: Annotated[list[AnyMessage], add_messages]  # 이미 상속받아서 쓸필요 없다
    lang: str

builder = StateGraph(state_schema=Mystate)

prompt_template = ChatPromptTemplate.from_messages([
    ('system', '너는 유능한 어시스턴트야. 너의 능력을 최대한 활용해서 답을 해봐. {lang}언어로 답해.'),
    MessagesPlaceholder(variable_name='messages'),
])

def simple_node(state: Mystate):
    chain = prompt_template | llm
    res = chain.invoke(state)
    return {'messages': res}

builder.add_node('simple_node', simple_node)
builder.add_edge(START, 'simple_node')
builder.add_edge('simple_node', END)

memory = MemorySaver()

graph = builder.compile(checkpointer=memory)

In [ ]:
config = {'configurable': {'thread_id': 'asdf123'}}

state = {
    # 'messages': [HumanMessage(content='안녕 내 이름은 밥이야')],
    # 'messages': [HumanMessage(content='내 이름이 뭐야?')],
    'messages': [HumanMessage(content='너는 어디에 있니?')],
    'lang': 'spanish'
}

res = graph.invoke(state, config)

for msg in res['messages']:
    msg.pretty_print()

================================ Human Message =================================

안녕 내 이름은 밥이야
================================== Ai Message ==================================

¡Hola, Bob! Encantado de conocerte. ¿En qué puedo ayudarte hoy?
================================ Human Message =================================

내 이름이 뭐야?
================================== Ai Message ==================================

Tu nombre es Bob.
================================ Human Message =================================

너는 어디에 있니?
================================== Ai Message ==================================

Soy un asistente virtual y no tengo una ubicación física, pero estoy aquí para ayudarte dondequiera que estés.


## 대화 기록 관리하기
대화 내역을 관리 안하면, LLM의 컨텍스트 윈도우(입력 최대치)를 넘어가버림. 더이상 채팅이 유지가 안된다.
- 과거를 잘라내거나
- 요약해서 정리하거나
- 등등

In [85]:
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage, trim_messages

trimmer = trim_messages(
    strategy='last',        # 최근 메시지에 대해서
    max_tokens=65,          # 최대 65토큰까지만 허용
    token_counter=llm,      # 내 llm 모델에 맞춰서 토큰 세기
    include_system=True,    # 정리할 때 시스템 메시지를 제외하라는 의미
    allow_partial=False,    # 메시지를 중간에서 자르지는 말 것
    start_on='human',       # 자른 메시지를 항상 사람부터 시작하게 하기
)

In [ ]:
messages = [
    SystemMessage(content="you're a good assistant"),
    
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]

In [86]:
trimmer.invoke(messages)

[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [95]:
class Mystate(MessagesState):
    lang: str

builder = StateGraph(state_schema=Mystate)

prompt_template = ChatPromptTemplate.from_messages([
    ('system', '너는 유능한 어시스턴트야. 너의 능력을 최대한 활용해서 답을 해봐. {lang}언어로 답해.'),
    MessagesPlaceholder(variable_name='messages'),
])

# 일단 전체 메시지 내용은 다 들어오는데, 노드 안에서 llm에 넣기 전에 가공, 답변 후 답변은 전체 메시지 내용에 추가함
trimmer = trim_messages(
    strategy='last',
    max_tokens=200,
    token_counter=llm,
    include_system=True,
    allow_partial=False,
    start_on='human',
)

def simple_node(state: Mystate):
    # 체인상 기존 메시지를 정리하고 프롬프트에 넣는게 맞다
    print(f'정리 전 메시지 개수: {len(state['messages'])}개')
    trimmed_messages = trimmer.invoke(state['messages'])
    print(f'정리 후 메시지 개수: {len(trimmed_messages)}개')
    
    state['messages'] = trimmed_messages
    chain = prompt_template | llm
    res = chain.invoke(state)
    
    return {'messages': res}

builder.add_node('simple_node', simple_node)
builder.add_edge(START, 'simple_node')
builder.add_edge('simple_node', END)

memory = MemorySaver()

graph = builder.compile(checkpointer=memory)

In [101]:
config = {'configurable': {'thread_id': 'asdf5678'}}

state = {
    'messages': [HumanMessage(content='아까 점심 메뉴 추천해준거 뭐였지?')],
    'lang': 'korean'
}

res = graph.invoke(state, config)

for msg in res['messages']:
    msg.pretty_print()

정리 전 메시지 개수: 11개
정리 후 메시지 개수: 3개
================================ Human Message =================================

안녕
================================== Ai Message ==================================

안녕하세요! 어떻게 도와드릴까요?
================================ Human Message =================================

슬슬 점심시간인데, 뭘 먹는게 좋을까?
================================== Ai Message ==================================

점심시간이라니 좋네요! 어떤 걸 좋아하시나요? 가벼운 샐러드나 샌드위치, 아니면 따뜻한 국이나 찌개 같은 것도 괜찮을 것 같아요. 혹은 국물이 시원한 냉면이나 쫄깃한 비빔밥도 좋아요. 시간과 장소, 그리고 기분에 따라 적절한 메뉴를 골라보면 더 좋을 것 같아요. 특별히 선호하는 음식이나 오늘의 컨디션에 맞춰 추천이 필요하시면 알려주세요!
================================ Human Message =================================

지금 좀 피곤하고, 졸린 상태야
================================== Ai Message ==================================

그렇다면 오늘은 가볍고 속 편한 음식을 추천드릴게요. 피곤하고 졸릴 때는 소화가 잘 되고 몸에 부담이 적은 음식이 좋아요. 예를 들어, 미소된장국이나 따뜻한 죽(예: 콩죽이나 곡물죽), 또는 가벼운 샌드위치나 토스트도 좋아요. 차 한잔이나 따뜻한 차와 함께 먹으면 기분도 좋아지고 몸도 편안해질 거예요. 조금 휴식을 취하면서 에너지를 충전하는 것도 중요하니, 느긋하게 드시길 권해드려요!
======

## 스트리밍

In [102]:
class Mystate(MessagesState):
    lang: str

builder = StateGraph(state_schema=Mystate)

prompt_template = ChatPromptTemplate.from_messages([
    ('system', '너는 유능한 어시스턴트야. 너의 능력을 최대한 활용해서 답을 해봐. {lang}언어로 답해.'),
    MessagesPlaceholder(variable_name='messages'),
])

trimmer = trim_messages(
    strategy='last',
    max_tokens=200,
    token_counter=llm,
    include_system=True,
    allow_partial=False,
    start_on='human',
)

def simple_node(state: Mystate):
    trimmed_messages = trimmer.invoke(state['messages'])    
    state['messages'] = trimmed_messages
    chain = prompt_template | llm
    res = chain.invoke(state)
    return {'messages': res}

builder.add_node('simple_node', simple_node)
builder.add_edge(START, 'simple_node')
builder.add_edge('simple_node', END)

memory = MemorySaver()

graph = builder.compile(checkpointer=memory)

In [105]:
config = {'configurable': {'thread_id': 'zxcv333'}}

state = {
    'messages': [HumanMessage(content='신라면이랑 너구리 중에 뭐가 더 맛있을까?')],
    'lang': '한국어'
}

for chunk, metadata in graph.stream(state, config=config, stream_mode='messages'):
    print(chunk.content, end='|')

|신|라|면|과| 너|구|리| 둘| 다| 인기| 많은| 라|면|이|지만|,| 어떤| 게| 더| 맛|있|다고| 단|정|짓|기| 어려|워|요|.| 각각|의| 특징|이| 있어서| 개인| 취|향|에| 따라| 다|를| 수| 있|거|든|요|.

|-| **|신|라|면|**|:| 매|콤|하고| 진|한| 국|물| 맛|이| 특징|으로|,| 매|운| 음|식을| 좋아|하는| 사람|들에게| 인|기가| 많|아요|.| 국|물|의| 깊|이|와| 풍|미|가| 좋아|서| 많은| 사람들이| 즐|겨| 먹|어요|.
|-| **|너|구|리|**|:| 해|물|과| 깔|끔|한| 맛|이| 나는| 국|물|로|,| 좀| 더| 담|백|하고| 부|드|러운| 맛|을| 좋아|하는| 분|들에게| 좋아|요|.| 특히|,| 간|이| 적|당|해서| 많은| 사람들이| 부담| 없이| 즐|기|기| 좋아|요|.

|결|국|,| 어떤| 맛|이| 더| 좋은|지는| 개인| 취|향|에| 따라| 다|르|니|,| 혹|시| 조금| 매|운| 것을| 좋아|한다|면| 신|라|면|,| 깔|끔|하고| 부|드|러운| 맛|을| 원|한다|면| 너|구|리를| 추천|드|려|요|!| 직접| 한| 번|씩| 먹|어|보|는| 것도| 좋은| 방법|일| 것| 같|아요|.||